In [ ]:
import unicodedata
import pandas as pd
import numpy as np


lista = ['        Novais Barros Ltda.', 'Freitas Monteiro          Ltda.', 'Abreu         Cirino                Ltda.']

df = pd.DataFrame({
        'teste':   lista,
        'coluna2': [1,2,3]
})

#remove espacos desnecessarios antes, entre e depois dos valores em formato string
df = df(lambda col: col.str.replace(r'\s+', ' ', regex=True).str.strip() if col.dtype == 'object' and col.apply(lambda x: isinstance(x, str)).any() else col)

import numpy as np

# Função para remover espaços desnecessários em todos os tipos de dados
def strip_all_types(col):
    # Converte a coluna para string, remove espaços extras e aplica strip
    col_stripped = col.astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
    
    # Converte de volta para o tipo original da coluna
    if np.issubdtype(col.dtype, np.integer):  # Se for inteiro
        return pd.to_numeric(col_stripped, errors='coerce').astype(col.dtype)
    elif np.issubdtype(col.dtype, np.floating):  # Se for float
        return pd.to_numeric(col_stripped, errors='coerce').astype(col.dtype)
    elif np.issubdtype(col.dtype, np.datetime64):  # Se for data
        return pd.to_datetime(col_stripped, errors='coerce')
    else:  # Se for string ou outro tipo
        return col_stripped

# Aplica a função a todas as colunas do DataFrame
df = df.apply(strip_all_types)

lista_formatada = []

# Processa cada item da lista
for item in lista:
    #espacos desnecessarios antes e depois da string
    item = item.strip()
    # Remove acentos
    item_sem_acentos = ''.join(
        letra for letra in unicodedata.normalize('NFD', item)
        if unicodedata.category(letra) != 'Mn'
    )
    
    # Remove caracteres especiais e pontos
    item_formatado = ''.join(
        letra for letra in item_sem_acentos
        if letra.isalnum() or letra.isspace()
    )
    
    # Adiciona o item formatado à lista
    lista_formatada.append(item_formatado)

# Exibe a lista formatada
display(df)




In [ ]:
'''SCRIPT PARA VERIFICAR SE DADOS DO DATAFRAME DE INSERCAO NO BANCO DE DADOS JA EXISTEM O USE PRECISAM SER ADICIONADOS'''
import pandas as pd
from datetime import date
from sqlalchemy import create_engine, VARCHAR, DATE, INTEGER, text
import numpy as np

# Configurações de conexão com o SQL Server
server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"
conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

#Função para remover espaços desnecessários em todos os tipos de dados
def strip_all_types(col):
    # Verifica se a coluna é do tipo datetime
    if np.issubdtype(col.dtype, np.datetime64):
        # Se for datetime, retorna a coluna original sem modificações
        return col
    
    col_stripped = col.astype(str).str.replace(r'\s+', ' ', regex=True).str.strip() #Converte a coluna para string, remove espaços extras e aplica strip
    
    #Converte de volta para o tipo original da coluna
    if np.issubdtype(col.dtype, np.integer):  # Se for inteiro
        return pd.to_numeric(col_stripped, errors='coerce').astype(col.dtype)
    elif np.issubdtype(col.dtype, np.floating):  # Se for float
        return pd.to_numeric(col_stripped, errors='coerce').astype(col.dtype)
    elif col.name in ["DATA_CRIACAO", "DATA_ATLZ"]:  # Se for uma coluna de data, Converte valores inválidos (como 'None') para NaT (Not a Time)
        return pd.to_datetime(col_stripped, errors='coerce')
    else:  # Se for string ou outro tipo
        return col_stripped

#Tabela de teste
#df_produto = pd.DataFrame({
#    "NOME": ["Hamburguer Seda2"],
#    "ID_CATEGORIA": [11],
#    "PRECO_CUSTO": [11.93],
#    "PRECO_UNIT": [15.17],
#    "ID_UNID_MED": [1],
#    "DATA_CRIACAO": ["2019/12/03"],
#    #"DATA_CRIACAO": pd.to_datetime(["2019-12-03"]).date,
#    "DATA_ATLZ": [None],
#    "VERSAO": [1]
#})

#Tabela e coluna a serem verificadas
tabela = "PRODUTO"
coluna = "NOME"

with engine.connect() as conn:
    # Consulta os valores existentes na coluna "NOME" da tabela "PRODUTO"
    query = text(f"SELECT {coluna} FROM {tabela}")
    lista_valores_existentes = pd.read_sql(query, conn)[coluna].tolist()
    print(lista_valores_existentes)
    print(len(lista_valores_existentes))

    # Filtra os valores que não existem na tabela
    df_produto_filtrado = df_produto[~df_produto["NOME"].isin(lista_valores_existentes)]
    display(df_produto_filtrado)

    # Se houver valores para adicionar
    if not df_produto_filtrado.empty:
        # Limpa os espaços desnecessários
        df_produto_filtrado = df_produto_filtrado.apply(strip_all_types)

        # Define os tipos de dados para a tabela PRODUTO
        tipo_dados_produto = {
            "NOME": VARCHAR(100),
            "ID_CATEGORIA": INTEGER,
            "PRECO_CUSTO": VARCHAR(20),
            "PRECO_UNIT": VARCHAR(20),
            "ID_UNID_MED": INTEGER,
            "DATA_CRIACAO": DATE,
            "DATA_ATLZ": DATE,
            "VERSAO": INTEGER,
        }

        # Exibe os dados que serão adicionados
        print("Dados a serem adicionados:")
        display(df_produto_filtrado)

        # Adiciona os dados ao banco de dados
        df_produto_filtrado.to_sql(
            name=tabela,
            con=engine,
            if_exists="append",
            index=False,
            dtype=tipo_dados_produto
        )

        print("Dados inseridos com sucesso.")
    else:
        print("Nenhum valor adicionado.")

# Fecha a conexão com o banco de dados
engine.dispose()